# Prep to have real selection

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
path = '.\..\..\dataset\PSD_to_Pandas\pump_6dB_00020406_full_spectrum_128.pkl'
df = pd.read_pickle(path)

# TODO make the freq thing internal...
frequ = df.columns[df.dtypes==np.float64]
descriptive = [label for label in df.columns[df.dtypes==np.object] if not 'abnormal' in label]
target = ['abnormal']
IDs = df.ID.unique()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df[df.ID==IDs[0]][df['abnormal']==0][frequ])
x = scaler.transform(df[df.ID==IDs[0]][frequ])
y = df.abnormal[df.ID==IDs[0]]
from sklearn.decomposition import PCA
n_prin_comp = 50
pca_test = PCA(n_components=n_prin_comp, svd_solver='full')
pca_test.fit(x)
X_pca = pca_test.transform(x)

def p_pca(X_pca, n, c):
    plt.scatter(X_pca[y==n][:,0], X_pca[y==n][:,1], color=c
                        , marker='+', alpha=.3, label=n)

In [ ]:
%matplotlib inline
%run ../cluster_visual_selector.py
#x_range = [-25,-15]
#y_range = [-20+7,-17+7]
x_range = [-35,-15]
y_range = [-13,-10]
box_idx = box_points(X_pca[:,0],X_pca[:,1], x_range, y_range)
p_pca(X_pca, 0, 'b')
p_pca(X_pca, 1, 'r')
box_outline_plot(x_range, y_range)
plt.legend()
plt.grid()
plt.xlim([-50,0])
plt.ylim([-30,0])
print(len(box_idx))
plt.show()

# Developement Zone

In [ ]:
BASE_FOLDER = '../../'
%run -i ..\feature_extractor\JupyterLoad_feature_extractor.py

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import os
from IPython.display import display, Markdown, Audio
import librosa
import librosa.display
import matplotlib.cm as cm

def audtion_display_path(filepath):
    display(Markdown('#### ' + filepath))

# just create audio player from IPython audio
def audition_listion(filepath, base_folder, stereo_ch=[0,4], time_plot_ch = 0, play=True):
    audtion_display_path(filepath + ' ch:' + str(stereo_ch))
    audio_ch  = \
            librosa.load(os.path.abspath(base_folder+filepath), sr=None, mono=False)
    
    if len(stereo_ch) == 1:
        stereo_ch.append(stereo_ch[0])
    elif stereo_ch == None:
        stereo_ch == [0,0]
    
    if time_plot_ch > -1:
        librosa.display.waveplot(audio_ch[0][time_plot_ch],
                                                 sr=audio_ch[1])
        plt.grid()
        
    if play:
        display(Audio(data=[audio_ch[0][stereo_ch[0]], \
                            audio_ch[0][stereo_ch[1]]], \
                  rate=audio_ch[1]))


class audition_akku_plot():
    def __init__(self, xlabel=x, ylabel='y'):
        self.y=[]
        self.name=[]
        self.x=[]
        self.hue_info=[]
        self.xlabel=xlabel
        self.ylabel=ylabel
        
    def plot(self,hue=True): # if hue is False plot legend by name
        # color map
        cmap_u = cm.get_cmap(name='viridis', lut=None)
        c_set = set(self.hue_info)
        c_set_l = list(c_set)
        c_set_c = np.linspace(0,1,len(c_set_l))
        print(c_set_c,c_set_l, len(self.x), len(self.x[2]), len(self.x[0]))
        for i in range(len(self.x)):
            #print(i)
            x = self.x[i]
            y = self.y[i]
            n = self.name[i]
            h = self.hue_info[i]
            if hue:
                if len(c_set_l)>1:
                    c = cmap_u(c_set_c[h])
                else:
                    c = 'b'
                
            plt.plot(x,y,label=h,color=c) # todo 
            
        plt.xlabel(self.xlabel) 
        plt.ylabel(self.ylabel) 
        plt.grid()
        #plt.legend()
        # https://matplotlib.org/3.1.1/gallery/text_labels_and_annotations/custom_legends.html

    
    def add_line(self,x,y,name,hue_info):
        self.y.append(y)
        self.x.append(x)
        self.name.append(name)
        self.hue_info.append(hue_info)

        
def audition_psd_plot(x,y,df_row, dB=True):  
    plt.plot(x,y)
    plt.grid()
    plt.title(f'psd {df_row.machine} id:{df_row.ID} snr:{df_row.SNR}')
    plt.xlabel('frequency [Hz]')
    if dB:
        plt.ylabel(' 10*log10(V**2)')
    else:
        plt.ylabel(' V**2 ')
        
def audition_psd_from_data(df_row, dB=True):
    #df_row=df.iloc[box_idx].iloc[0]
    y = df_row[[type(e) is float for e in df_row.index]]
    x = y.index.values.astype('float')
    if dB:
        y = 10*np.log10(y.values.astype('float'))
        
    return x,y

def audition_psd_from_data_plot(df_row, dB=True):
    x, y = audition_psd_from_data(df_row, dB)
    audition_psd_plot(x,y,df_row, dB)
        
#def audition_spectra_plot_grid()

# create a report list with Ipython players
def make_audition_list(df_select,
                       base_folder=r'./',
                       no_section = False,
                       listen=True,
                       listen_every=1, # buggy for now
                       listen_stereo_ch=[0,4],
                       time_plot_ch = -1, # turn of by setting -1
                       psd_from_data = False,
                       psd_from_data_dB = True,
                       fft_from_data = False, # not there
                       Combine_psdfft = False, # maybe changhe to have both
                       Combine_psdfft_hue = 'abnormal', # not fully implemented legend missing
                       Combine_psdfft_hue_colormap = None, # not forwarded
                       mel_from_data = False,    # not there
                       mel_create = False,
                       mel_create_para = {'n_mels':64, 
                                     'n_fft':1024,
                                     'power':2.0,
                                    'hop_length':512}, # expects mel paramters
                       ):
    
    # akku classes
    if Combine_psdfft:
        psd_akkuplot = audition_akku_plot('freq. [Hz]', 'V^2') # TODO dB 
    
    for row in range(len(df_select.index)):
        
        # Dispaly main section and possible audio player
        if listen:
            if row%listen_every==0:
            
                audition_listion(df_select.path.iloc[row], 
                             base_folder, 
                             listen_stereo_ch, 
                             time_plot_ch)
            plt.show()
        else: 

            if time_plot_ch > -1:
                audition_listion(df_select.path.iloc[row], 
                             base_folder, 
                             listen_stereo_ch, 
                             time_plot_ch,play=False)
                plt.show()
            else:
                if not no_section:
                    audtion_display_path(df_select.path.iloc[row])
            
        df_row = df_select.iloc[row]    
        # create single psd for every section
        
        if psd_from_data and not Combine_psdfft:
            if row%listen_every==0:
                audition_psd_from_data_plot(df_select.iloc[row], psd_from_data_dB)
                plt.show()
        
        if mel_create:
            fe_mel = feature_extractor_mel(base_folder)
            fe_mel.create_from_wav(df_select.path.iloc[row])
            fe_mel.plot()
            plt.show()
        
        
        # akku psd
        if psd_from_data and  Combine_psdfft:  
            hue_info = df_row[Combine_psdfft_hue]
            x, y = audition_psd_from_data(df_row, psd_from_data_dB)
            # TODO name ...
            psd_akkuplot.add_line(x,y,'#',hue_info)
          
        
        
    if psd_from_data and  Combine_psdfft:
        psd_akkuplot.plot() # TODO react on hue
        

In [ ]:
make_audition_list(df.iloc[box_idx][df.iloc[box_idx].abnormal==0].iloc[0:1],'../../',
                   listen=True, time_plot_ch = 0,
                   psd_from_data=True, 
                   Combine_psdfft=False, 
                   listen_every = 1, 
                   no_section=False,
                  mel_create=True)
make_audition_list(df.iloc[box_idx][df.iloc[box_idx].abnormal==1].iloc[0:1],'../../',
                   listen=True, time_plot_ch = 0,
                   psd_from_data=True, 
                   Combine_psdfft=False, 
                   listen_every = 1, 
                   no_section=False,
                  mel_create=True)

In [ ]:
#### clear output !!!!!!!!!